<a href="https://colab.research.google.com/github/elemi10/7506-TP-Org-de-datos/blob/7506-Trabajo-Practico-1/TP1_Real_or_Not%3F_NLP_with_Disaster_Tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **TP1 - Real or Not?**

###  **Análisis exploratorio**

![Banner_TP1.png](https://drive.google.com/uc?id=1BPA2RF1SDm9bTs1xZfVUa1VQn932E3wr)


In [0]:
#IMPORT FILES FROM DRIVE INTO GOOGLE-COLAB:

#STEP-1: Import Libraries

# Code to read csv file into colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
#STEP-2: Autheticate E-Mail ID

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
#STEP-3: Get File from Drive using file-ID

#2.1 Get the file
downloaded = drive.CreateFile({'id':'1RAGDjlzJ6spO5Sq8_x3UTIvxLhKAUBEt'}) # replace the id with id of file you want to access
downloaded.GetContentFile('train.csv') 

#downloaded = drive.CreateFile({'id':'17pAgG9oJRK1bAFWRKkp96__zicG6yUmy'}) # replace the id with id of file you want to access
#downloaded.GetContentFile('test.csv') 

#downloaded = drive.CreateFile({'id':'1u8v51BT7FZggIRD-eo0dQno--0wlxIhA'}) # replace the id with id of file you want to access
#downloaded.GetContentFile('sample_submission.csv') 


In [0]:
#STEP-4: Import Libraries

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import colors
import datetime as dt
import scipy as sp
#from sklearn.cross_validation import train_test_split -- no levanta
%matplotlib inline
plt.rcParams['figure.figsize'] = (16,9)
plt.style.use('default') # mejoramos esteticamente un poco los gráficos en matplotlib
sns.set(style='whitegrid') # seteando tipo de grid en seaborn

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_selection import SelectKBest
from sklearn import datasets, linear_model
from sklearn.feature_extraction.text import CountVectorizer
#from sklearn.cross_validation import train_test_split --> No levanta el feature de la libreria
from sklearn.naive_bayes import MultinomialNB

### Levantamos archivo **train.csv**


In [280]:
df_train = pd.read_csv(r'train.csv', usecols=['keyword','location','text','target']) 
df_train.head(10)

,keyword,location,text,target
0,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
5,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,1
6,NaN,NaN,#flood #disaster Heavy rain causes flash flood...,1
7,NaN,NaN,I'm on top of the hill and I can see a fire in...,1
8,NaN,NaN,There's an emergency evacuation happening now ...,1
9,NaN,NaN,I'm afraid that the tornado is coming to our a...,1


In [0]:
# exporto a excel
df_train.to_excel('df_train_preprocesado.xlsx')

In [282]:
df_train.info()
# las variables 'keyword' y 'location' estan incompletas

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   keyword   7552 non-null   object
 1   location  5080 non-null   object
 2   text      7613 non-null   object
 3   target    7613 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 238.0+ KB


In [0]:
#df_train['target'].value_counts()

### Creamos una variable para **contar total caracteres** del tweet

In [284]:
# Creo una columna con el cálculo de las palabras que tiene la columna 'text' y la llamo 'caracteres_tweet'. 
# Despues consulto con un head()
df_train['caracteres_tweet']=df_train['text'].str.len()
df_train.head(20)

,keyword,location,text,target,caracteres_tweet
0,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,69
1,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,38
2,NaN,NaN,All residents asked to 'shelter in place' are ...,1,133
3,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,65
4,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,88
5,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,1,110
6,NaN,NaN,#flood #disaster Heavy rain causes flash flood...,1,95
7,NaN,NaN,I'm on top of the hill and I can see a fire in...,1,59
8,NaN,NaN,There's an emergency evacuation happening now ...,1,79
9,NaN,NaN,I'm afraid that the tornado is coming to our a...,1,52


In [285]:
# Verificamos la existencia de valores nulos
df_train.isnull().any()

keyword              True
location             True
text                False
target              False
caracteres_tweet    False
dtype: bool

In [286]:
df_train.isnull().sum()

keyword               61
location            2533
text                   0
target                 0
caracteres_tweet       0
dtype: int64

In [287]:
df_train.describe()

,target,caracteres_tweet
count,7613.00000,7613.000000
mean,0.42966,101.037436
std,0.49506,33.781325
min,0.00000,7.000000
25%,0.00000,78.000000
50%,0.00000,107.000000
75%,1.00000,133.000000
max,1.00000,157.000000


In [288]:
# cuento total de tweets analizados
total_tweets=len(df_train)
total_tweets

7613

In [289]:
# verifico cuantos son verdaderos
tweets_verdaderos=(df_train['target']==1).sum()
tweets_verdaderos


3271

In [290]:
# verifico cuantos son fakes
tweets_falsos=(df_train['target']==0).sum()
tweets_falsos

4342

In [291]:
# calculo la probabilidad (casos favorables/sobre casos posibles) → en este casos es equivalente .mean()
probabilidad_tweet_verdadero = np.round((tweets_verdaderos)/(total_tweets),decimals=2)
probabilidad_tweet_verdadero

0.43

In [292]:
# calculo la probabilidad (casos favorables/sobre casos posibles) → en este casos es equivalente .mean()
probabilidad_tweet_falso = np.round((tweets_falsos)/(total_tweets),decimals=2)
probabilidad_tweet_falso

0.57

### **Iniciamos proceso de limpieza de datos**

#### **Verificación de nulos y normalización DF**



In [293]:
# Verifico que no haya instancias nulas o filas completas nulas
df_train=df_train.dropna(how="all")
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   keyword           7552 non-null   object
 1   location          5080 non-null   object
 2   text              7613 non-null   object
 3   target            7613 non-null   int64 
 4   caracteres_tweet  7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 356.9+ KB


In [294]:
# Normalizo el dataframe los registros NaN con texto
df_train=df_train.fillna({'keyword': 'sin keyword',\
                   'location': 'sin location'})
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   keyword           7613 non-null   object
 1   location          7613 non-null   object
 2   text              7613 non-null   object
 3   target            7613 non-null   int64 
 4   caracteres_tweet  7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 356.9+ KB


In [0]:
# Eliminamos los tweets duplicados considerando que su cantidad no tiene impacto en el análisis.
df_train=df_train.drop_duplicates('text')

In [296]:
df_train
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7503 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   keyword           7503 non-null   object
 1   location          7503 non-null   object
 2   text              7503 non-null   object
 3   target            7503 non-null   int64 
 4   caracteres_tweet  7503 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 351.7+ KB


## Preprocesamiento de datos: **Tokenización**

In [0]:
import string
punctuation = set(string.punctuation)

def tokenize(sentence):
    tokens = []
    for token in sentence.split():
        new_token = []
        for character in token:
            if character not in punctuation:
                new_token.append(character.lower())
        if new_token:
            tokens.append("".join(new_token))
    return tokens

In [0]:
# type(tokens)

In [0]:
df_train['text_vector']=df_train['text'].apply(tokenize)

In [300]:
df_train.head(5)

,keyword,location,text,target,caracteres_tweet,text_vector
0,sin keyword,sin location,Our Deeds are the Reason of this #earthquake M...,1,69,"[our, deeds, are, the, reason, of, this, earth..."
1,sin keyword,sin location,Forest fire near La Ronge Sask. Canada,1,38,"[forest, fire, near, la, ronge, sask, canada]"
2,sin keyword,sin location,All residents asked to 'shelter in place' are ...,1,133,"[all, residents, asked, to, shelter, in, place..."
3,sin keyword,sin location,"13,000 people receive #wildfires evacuation or...",1,65,"[13000, people, receive, wildfires, evacuation..."
4,sin keyword,sin location,Just got sent this photo from Ruby #Alaska as ...,1,88,"[just, got, sent, this, photo, from, ruby, ala..."


In [301]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7503 entries, 0 to 7612
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   keyword           7503 non-null   object
 1   location          7503 non-null   object
 2   text              7503 non-null   object
 3   target            7503 non-null   int64 
 4   caracteres_tweet  7503 non-null   int64 
 5   text_vector       7503 non-null   object
dtypes: int64(2), object(4)
memory usage: 410.3+ KB


In [0]:
#from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
#from sklearn.feature_extraction import stop_words
#print(stop_words.ENGLISH_STOP_WORDS)

In [303]:
import nltk
nltk.download('stopwords')

#from nltk.corpus import stopwords
#from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [304]:
from nltk.corpus import stopwords
#from nltk.tokenize import word_tokenize
stopwords.fileids()

['arabic',
 'azerbaijani',
 'danish',
 'dutch',
 'english',
 'finnish',
 'french',
 'german',
 'greek',
 'hungarian',
 'indonesian',
 'italian',
 'kazakh',
 'nepali',
 'norwegian',
 'portuguese',
 'romanian',
 'russian',
 'slovene',
 'spanish',
 'swedish',
 'tajik',
 'turkish']

In [0]:
stop=stopwords.words('english')

In [0]:
def filtro(text_vector):
  text_vector_filtrado = []
  for palabra in text_vector:
    if palabra not in stop:
      text_vector_filtrado.append(palabra)
  return(text_vector_filtrado)

In [0]:
# type(text_vector_filtrado)

In [0]:
df_train['text_vector_filtrado']=df_train['text_vector'].apply(filtro)

In [309]:
df_train['text_vector_filtrado'].head()

0    [deeds, reason, earthquake, may, allah, forgiv...
1        [forest, fire, near, la, ronge, sask, canada]
2    [residents, asked, shelter, place, notified, o...
3    [13000, people, receive, wildfires, evacuation...
4    [got, sent, photo, ruby, alaska, smoke, wildfi...
Name: text_vector_filtrado, dtype: object

In [310]:
df_train['elem_vector']=df_train['text_vector'].str.len()
df_train.head()

,keyword,location,text,target,caracteres_tweet,text_vector,text_vector_filtrado,elem_vector
0,sin keyword,sin location,Our Deeds are the Reason of this #earthquake M...,1,69,"[our, deeds, are, the, reason, of, this, earth...","[deeds, reason, earthquake, may, allah, forgiv...",13
1,sin keyword,sin location,Forest fire near La Ronge Sask. Canada,1,38,"[forest, fire, near, la, ronge, sask, canada]","[forest, fire, near, la, ronge, sask, canada]",7
2,sin keyword,sin location,All residents asked to 'shelter in place' are ...,1,133,"[all, residents, asked, to, shelter, in, place...","[residents, asked, shelter, place, notified, o...",22
3,sin keyword,sin location,"13,000 people receive #wildfires evacuation or...",1,65,"[13000, people, receive, wildfires, evacuation...","[13000, people, receive, wildfires, evacuation...",8
4,sin keyword,sin location,Just got sent this photo from Ruby #Alaska as ...,1,88,"[just, got, sent, this, photo, from, ruby, ala...","[got, sent, photo, ruby, alaska, smoke, wildfi...",16


In [311]:
df_train['elem_vector_filtrado']=df_train['text_vector_filtrado'].str.len()
df_train.head()

,keyword,location,text,target,caracteres_tweet,text_vector,text_vector_filtrado,elem_vector,elem_vector_filtrado
0,sin keyword,sin location,Our Deeds are the Reason of this #earthquake M...,1,69,"[our, deeds, are, the, reason, of, this, earth...","[deeds, reason, earthquake, may, allah, forgiv...",13,7
1,sin keyword,sin location,Forest fire near La Ronge Sask. Canada,1,38,"[forest, fire, near, la, ronge, sask, canada]","[forest, fire, near, la, ronge, sask, canada]",7,7
2,sin keyword,sin location,All residents asked to 'shelter in place' are ...,1,133,"[all, residents, asked, to, shelter, in, place...","[residents, asked, shelter, place, notified, o...",22,11
3,sin keyword,sin location,"13,000 people receive #wildfires evacuation or...",1,65,"[13000, people, receive, wildfires, evacuation...","[13000, people, receive, wildfires, evacuation...",8,7
4,sin keyword,sin location,Just got sent this photo from Ruby #Alaska as ...,1,88,"[just, got, sent, this, photo, from, ruby, ala...","[got, sent, photo, ruby, alaska, smoke, wildfi...",16,9


In [312]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7503 entries, 0 to 7612
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   keyword               7503 non-null   object
 1   location              7503 non-null   object
 2   text                  7503 non-null   object
 3   target                7503 non-null   int64 
 4   caracteres_tweet      7503 non-null   int64 
 5   text_vector           7503 non-null   object
 6   text_vector_filtrado  7503 non-null   object
 7   elem_vector           7503 non-null   int64 
 8   elem_vector_filtrado  7503 non-null   int64 
dtypes: int64(4), object(5)
memory usage: 586.2+ KB


### Preproceso y categorización de la variable ***Keyword***

In [0]:
#1 "Limpiamos" la serie de caracteres especiales, sustituyendolos
df_train['keyword']=df_train['keyword'].str.replace('%20','_')

In [314]:
#1 Verificamos actualización
df_train['keyword'].unique()

array(['sin keyword', 'ablaze', 'accident', 'aftershock',
       'airplane_accident', 'ambulance', 'annihilated', 'annihilation',
       'apocalypse', 'armageddon', 'army', 'arson', 'arsonist', 'attack',
       'attacked', 'avalanche', 'battle', 'bioterror', 'bioterrorism',
       'blaze', 'blazing', 'bleeding', 'blew_up', 'blight', 'blizzard',
       'blood', 'bloody', 'blown_up', 'body_bag', 'body_bagging',
       'body_bags', 'bomb', 'bombed', 'bombing', 'bridge_collapse',
       'buildings_burning', 'buildings_on_fire', 'burned', 'burning',
       'burning_buildings', 'bush_fires', 'casualties', 'casualty',
       'catastrophe', 'catastrophic', 'chemical_emergency', 'cliff_fall',
       'collapse', 'collapsed', 'collide', 'collided', 'collision',
       'crash', 'crashed', 'crush', 'crushed', 'curfew', 'cyclone',
       'damage', 'danger', 'dead', 'death', 'deaths', 'debris', 'deluge',
       'deluged', 'demolish', 'demolished', 'demolition', 'derail',
       'derailed', 'derailmen

In [315]:
#1 Listo para generar un diccionario y traducir la serie 'keyword'
keyword_list=df_train['keyword'].unique().tolist()
print(keyword_list)

['sin keyword', 'ablaze', 'accident', 'aftershock', 'airplane_accident', 'ambulance', 'annihilated', 'annihilation', 'apocalypse', 'armageddon', 'army', 'arson', 'arsonist', 'attack', 'attacked', 'avalanche', 'battle', 'bioterror', 'bioterrorism', 'blaze', 'blazing', 'bleeding', 'blew_up', 'blight', 'blizzard', 'blood', 'bloody', 'blown_up', 'body_bag', 'body_bagging', 'body_bags', 'bomb', 'bombed', 'bombing', 'bridge_collapse', 'buildings_burning', 'buildings_on_fire', 'burned', 'burning', 'burning_buildings', 'bush_fires', 'casualties', 'casualty', 'catastrophe', 'catastrophic', 'chemical_emergency', 'cliff_fall', 'collapse', 'collapsed', 'collide', 'collided', 'collision', 'crash', 'crashed', 'crush', 'crushed', 'curfew', 'cyclone', 'damage', 'danger', 'dead', 'death', 'deaths', 'debris', 'deluge', 'deluged', 'demolish', 'demolished', 'demolition', 'derail', 'derailed', 'derailment', 'desolate', 'desolation', 'destroy', 'destroyed', 'destruction', 'detonate', 'detonation', 'devastat

In [316]:
#1 Genero otra lista - pero en español
keyword_list_esp=['sin_keyword','en llamas','accidente','replica','accidente_avion','ambulancia','aniquilado','aniquilacion','apocalipse','armageddon','ejercito','incendio_intencional','piromano','ataque','atacado','avalancha','combate/batalla/lucha','bioterror','bioterrorismo','resplandecer/arder','resplandeciente/ardiente','hemorragia','exploto','plaga','ventisca','sangre','sangriento','estallido','bolsa_de_cadaver','embolsado_de_cadaver','bolsas_de_cadaver','bomba','bombardeado','bombardeo','colapso_de_puente','incendio_de_edificios','edificios_en_llamas','quemado','quemando/incendiando','incendio_edificios','incendio_de_matorrales/arbustos','bajas/muertes/perdidas','baja/muerte/perdida','catastrofe','catastrofico','emergencia_quimica','caida_acantilado','colapso','colapsado','colisionar','colisiono','colision','choque','estrellado','aplastamiento','aplastado','toque_de_queda','ciclon','da¤o','peligro','muerto','muerte','muertes','escombros','diluvio','inundado','demoler','demolido','demolicion','descarrilar','descarrilado','descarrilamiento','desolado','desolacion','destruir','destruido','destruccion','detonado','detonacion','devastado','devastacion','desastre','dislacado/desplazado','sequia','ahogar','ahogado','ahogamiento/ahogandose','tormenta_polvo','terremoto','electrocutar','electrocutado','emergencia','plan_de_emergencia','servicio_de_emergencia','engullido','epicentro','evacuar','evacuado','evacuacion','explotar','exploto/detonado/estallado','explosion','testigo_ocular','hambruna','fatal','fatalidades','fatalidades','miedo','fuego','camion_bombero','primeros_en_responder','llamas','aplastado','inundacion','inundaciones','inundaciones','incendio_forestal','incendio_forestales','granizo','tormenta_granizo','da¤o','peligro','peligroso','ola_de_calor','infierno','secuestro','secuestrador','secuestro','rehen','rehenes','huracan','herido/lesionado','heridas/lesiones','herida/lesion','inundado','inundacion','deslisamiento_tierra','lava','rayo','golpe_fuerte/estampido_fuerte','asesinato_en_serie','asesino_en_serie','masacre','caos','fusion','militar','alud_de_lodo','desastre_natural','desastre_nuclear','reactor_nuclear','obliterar/destruir/eliminar','borrado/destruido/eliminado','obliteracion/destruccion/eliminacion','derrame_de_petroleo','brote','pandemonio/confusion/caos','panico','tener_panico','polic¡a','cuarentena','en_cuarentena','emergencia_radiacion','tormenta','arrasado','refugiados','rescate','rescatado','rescatadores','disturbio','disturbios','escombros','ruina','tormenta_de_arena','grito','gritando','gritos','sismico','sumidero/pozo','hundiendose','sirena','sirenas','humo','tormenta_de_nieve','tormenta','camilla','fallo_estructural','bomba_suicida','suicida','bombardeo_suicida','hundido','sobrevivir','sobrevivio','sobreviviente','terrorismo','terrorista','amenaza','trueno','tormenta_electrica','tornado','tragedia','atrapado','trauma','traumatizado','problema','tsunami','tornado/torbellino','tifon','transtorno','tormenta_violenta','volcan','zona_de_guerra','arma','armas','torbellino','incendios_forestales','incendio_forestal','tormenta_de_viento','moretoneado','moretones','naufragio','restos','naufrago']
print(keyword_list_esp)

['sin_keyword', 'en llamas', 'accidente', 'replica', 'accidente_avion', 'ambulancia', 'aniquilado', 'aniquilacion', 'apocalipse', 'armageddon', 'ejercito', 'incendio_intencional', 'piromano', 'ataque', 'atacado', 'avalancha', 'combate/batalla/lucha', 'bioterror', 'bioterrorismo', 'resplandecer/arder', 'resplandeciente/ardiente', 'hemorragia', 'exploto', 'plaga', 'ventisca', 'sangre', 'sangriento', 'estallido', 'bolsa_de_cadaver', 'embolsado_de_cadaver', 'bolsas_de_cadaver', 'bomba', 'bombardeado', 'bombardeo', 'colapso_de_puente', 'incendio_de_edificios', 'edificios_en_llamas', 'quemado', 'quemando/incendiando', 'incendio_edificios', 'incendio_de_matorrales/arbustos', 'bajas/muertes/perdidas', 'baja/muerte/perdida', 'catastrofe', 'catastrofico', 'emergencia_quimica', 'caida_acantilado', 'colapso', 'colapsado', 'colisionar', 'colisiono', 'colision', 'choque', 'estrellado', 'aplastamiento', 'aplastado', 'toque_de_queda', 'ciclon', 'da¤o', 'peligro', 'muerto', 'muerte', 'muertes', 'escomb

In [317]:
#1 Genero el diccionario eng-esp de la serie 'Keyword'
keyword_dicc={}
y=0
for x in keyword_list:
  keyword_dicc[x]=keyword_list_esp[y]
  y=y+1
  #print(keyword_dicc)

print(keyword_dicc)

{'sin keyword': 'sin_keyword', 'ablaze': 'en llamas', 'accident': 'accidente', 'aftershock': 'replica', 'airplane_accident': 'accidente_avion', 'ambulance': 'ambulancia', 'annihilated': 'aniquilado', 'annihilation': 'aniquilacion', 'apocalypse': 'apocalipse', 'armageddon': 'armageddon', 'army': 'ejercito', 'arson': 'incendio_intencional', 'arsonist': 'piromano', 'attack': 'ataque', 'attacked': 'atacado', 'avalanche': 'avalancha', 'battle': 'combate/batalla/lucha', 'bioterror': 'bioterror', 'bioterrorism': 'bioterrorismo', 'blaze': 'resplandecer/arder', 'blazing': 'resplandeciente/ardiente', 'bleeding': 'hemorragia', 'blew_up': 'exploto', 'blight': 'plaga', 'blizzard': 'ventisca', 'blood': 'sangre', 'bloody': 'sangriento', 'blown_up': 'estallido', 'body_bag': 'bolsa_de_cadaver', 'body_bagging': 'embolsado_de_cadaver', 'body_bags': 'bolsas_de_cadaver', 'bomb': 'bomba', 'bombed': 'bombardeado', 'bombing': 'bombardeo', 'bridge_collapse': 'colapso_de_puente', 'buildings_burning': 'incendio_

In [318]:
#1 Mapeo los valores de las key del diccionario keyword_dicc
df_train['keyword_esp']=df_train['keyword'].map(keyword_dicc)
df_train.head(80)

,keyword,location,text,target,caracteres_tweet,text_vector,text_vector_filtrado,elem_vector,elem_vector_filtrado,keyword_esp
0,sin keyword,sin location,Our Deeds are the Reason of this #earthquake M...,1,69,"[our, deeds, are, the, reason, of, this, earth...","[deeds, reason, earthquake, may, allah, forgiv...",13,7,sin_keyword
1,sin keyword,sin location,Forest fire near La Ronge Sask. Canada,1,38,"[forest, fire, near, la, ronge, sask, canada]","[forest, fire, near, la, ronge, sask, canada]",7,7,sin_keyword
2,sin keyword,sin location,All residents asked to 'shelter in place' are ...,1,133,"[all, residents, asked, to, shelter, in, place...","[residents, asked, shelter, place, notified, o...",22,11,sin_keyword
3,sin keyword,sin location,"13,000 people receive #wildfires evacuation or...",1,65,"[13000, people, receive, wildfires, evacuation...","[13000, people, receive, wildfires, evacuation...",8,7,sin_keyword
4,sin keyword,sin location,Just got sent this photo from Ruby #Alaska as ...,1,88,"[just, got, sent, this, photo, from, ruby, ala...","[got, sent, photo, ruby, alaska, smoke, wildfi...",16,9,sin_keyword
...,...,...,...,...,...,...,...,...,...,...
76,accident,Norf Carolina,'By accident' they knew what was gon happen ht...,0,67,"[by, accident, they, knew, what, was, gon, hap...","[accident, knew, gon, happen, httpstcoysxun5vceh]",9,5,accidente
77,accident,"San Mateo County, CA",Traffic accident N CABRILLO HWY/MAGELLAN AV MI...,1,67,"[traffic, accident, n, cabrillo, hwymagellan, ...","[traffic, accident, n, cabrillo, hwymagellan, ...",9,9,accidente
78,accident,North Carolina,I-77 Mile Marker 31 to 40 South Mooresville I...,1,95,"[i77, mile, marker, 31, to, 40, south, mooresv...","[i77, mile, marker, 31, 40, south, mooresville...",16,14,accidente
79,accident,"Njoro, Kenya",the pastor was not in the scene of the acciden...,1,91,"[the, pastor, was, not, in, the, scene, of, th...","[pastor, scene, accidentwho, owner, range, rover]",17,6,accidente


In [319]:
#2 Categorizo sobre la variable 'keyword_esp'
pd.get_dummies(df_train['keyword_esp'], drop_first=True).shape

(7503, 210)

In [320]:
#2 Vemos el top 20 de las categorias mas frecuentes para la serie keyword 
df_train['keyword_esp'].value_counts().sort_values(ascending=False).head(20)

fatalidades           82
da¤o                  82
inundaciones          72
peligro               70
tormenta              69
escombros             65
aplastado             65
secuestro             64
inundado              62
sin_keyword           56
incendio_forestal     52
inundacion            45
armageddon            42
diluvio               42
bolsas_de_cadaver     41
sirena                40
miedo                 40
tornado/torbellino    40
evacuar               40
tormenta_de_viento    40
Name: keyword_esp, dtype: int64

In [321]:
#2 Hacemos un groupby de las categorias keyword con tweets verdaderos y falsos ('target')
df_agrupado=df_train.groupby(['keyword_esp','target']).size().sort_values(ascending=False)
df_agrupado

keyword_esp               target
da¤o                      0         59
aplastado                 0         56
peligro                   0         55
inundaciones              1         51
inundado                  0         49
                                    ..
bolsas_de_cadaver         1          1
derrame_de_petroleo       0          1
bolsa_de_cadaver          1          1
resplandeciente/ardiente  1          1
ruina                     1          1
Length: 419, dtype: int64

In [322]:
df_train['keyword_esp'].value_counts().nlargest(20)

fatalidades           82
da¤o                  82
inundaciones          72
peligro               70
tormenta              69
escombros             65
aplastado             65
secuestro             64
inundado              62
sin_keyword           56
incendio_forestal     52
inundacion            45
diluvio               42
armageddon            42
bolsas_de_cadaver     41
sirena                40
miedo                 40
tornado/torbellino    40
evacuar               40
tormenta_de_viento    40
Name: keyword_esp, dtype: int64

In [323]:
df_agrupado

keyword_esp               target
da¤o                      0         59
aplastado                 0         56
peligro                   0         55
inundaciones              1         51
inundado                  0         49
                                    ..
bolsas_de_cadaver         1          1
derrame_de_petroleo       0          1
bolsa_de_cadaver          1          1
resplandeciente/ardiente  1          1
ruina                     1          1
Length: 419, dtype: int64

In [324]:
df_train_by_keyword_esp=df_agrupado.groupby(['keyword_esp','target']).count()
df_train_by_keyword_esp=df_train_by_keyword_esp.reset_index()
df_train_by_keyword_esp

,keyword_esp,target,0
0,accidente,0,1
1,accidente,1,1
2,accidente_avion,0,1
3,accidente_avion,1,1
4,ahogado,0,1
...,...,...,...
414,ventisca,1,1
415,volcan,0,1
416,volcan,1,1
417,zona_de_guerra,0,1


In [325]:
df_agrupado2=df_train.groupby(['keyword_esp']).agg({'target':['count'],'caracteres_tweet':['max','min','mean']})
df_agrupado2

target caracteres_tweet                
                        count              max min        mean
keyword_esp                                                   
accidente                  35              142  35  100.085714
accidente_avion            34              140  26  105.529412
ahogado                    38              143  31   98.131579
ahogamiento/ahogandose     34              139  27   92.794118
ahogar                     32              145  23   84.437500
...                       ...              ...  ..         ...
trueno                     38              146  10   72.868421
tsunami                    34              148  18   85.764706
ventisca                   37              140  20   71.891892
volcan                     26              143  30  110.692308
zona_de_guerra             24              140  31   81.500000

[211 rows x 4 columns]

In [0]:
#df_agrupado.loc[df_agrupado[('keyword_esp','count')]>5,:].sort_values(('keyword_esp','count'),ascending=False)

**One-Hot encoding**

In [327]:
df_train

,keyword,location,text,target,caracteres_tweet,text_vector,text_vector_filtrado,elem_vector,elem_vector_filtrado,keyword_esp
0,sin keyword,sin location,Our Deeds are the Reason of this #earthquake M...,1,69,"[our, deeds, are, the, reason, of, this, earth...","[deeds, reason, earthquake, may, allah, forgiv...",13,7,sin_keyword
1,sin keyword,sin location,Forest fire near La Ronge Sask. Canada,1,38,"[forest, fire, near, la, ronge, sask, canada]","[forest, fire, near, la, ronge, sask, canada]",7,7,sin_keyword
2,sin keyword,sin location,All residents asked to 'shelter in place' are ...,1,133,"[all, residents, asked, to, shelter, in, place...","[residents, asked, shelter, place, notified, o...",22,11,sin_keyword
3,sin keyword,sin location,"13,000 people receive #wildfires evacuation or...",1,65,"[13000, people, receive, wildfires, evacuation...","[13000, people, receive, wildfires, evacuation...",8,7,sin_keyword
4,sin keyword,sin location,Just got sent this photo from Ruby #Alaska as ...,1,88,"[just, got, sent, this, photo, from, ruby, ala...","[got, sent, photo, ruby, alaska, smoke, wildfi...",16,9,sin_keyword
...,...,...,...,...,...,...,...,...,...,...
7604,sin keyword,sin location,#WorldNews Fallen powerlines on G:link tram: U...,1,136,"[worldnews, fallen, powerlines, on, glink, tra...","[worldnews, fallen, powerlines, glink, tram, u...",19,13,sin_keyword
7605,sin keyword,sin location,on the flip side I'm at Walmart and there is a...,1,114,"[on, the, flip, side, im, at, walmart, and, th...","[flip, side, im, walmart, bomb, everyone, evac...",26,10,sin_keyword
7606,sin keyword,sin location,Suicide bomber kills 15 in Saudi security site...,1,121,"[suicide, bomber, kills, 15, in, saudi, securi...","[suicide, bomber, kills, 15, saudi, security, ...",16,15,sin_keyword
7608,sin keyword,sin location,Two giant cranes holding a bridge collapse int...,1,83,"[two, giant, cranes, holding, a, bridge, colla...","[two, giant, cranes, holding, bridge, collapse...",11,9,sin_keyword


In [328]:
df_train_dummies=pd.get_dummies(df_train.keyword)
df_train_dummies.head()

,ablaze,accident,aftershock,airplane_accident,ambulance,annihilated,annihilation,apocalypse,armageddon,army,arson,arsonist,attack,attacked,avalanche,battle,bioterror,bioterrorism,blaze,blazing,bleeding,blew_up,blight,blizzard,blood,bloody,blown_up,body_bag,body_bagging,body_bags,bomb,bombed,bombing,bridge_collapse,buildings_burning,buildings_on_fire,burned,burning,burning_buildings,bush_fires,...,snowstorm,storm,stretcher,structural_failure,suicide_bomb,suicide_bomber,suicide_bombing,sunk,survive,survived,survivors,terrorism,terrorist,threat,thunder,thunderstorm,tornado,tragedy,trapped,trauma,traumatised,trouble,tsunami,twister,typhoon,upheaval,violent_storm,volcano,war_zone,weapon,weapons,whirlwind,wild_fires,wildfire,windstorm,wounded,wounds,wreck,wreckage,wrecked
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
df_train_merged = pd.concat([df_train,df_train_dummies],axis='columns')
#pd.options.display.max_columns = 80 -- quise que se muestren todas las columnas pero no hizo nada
# df_train_merged.columns.tolist() -- para ver todas las columnas

In [330]:
df_train_merged_final=df_train_merged.drop(['keyword_esp','sin keyword'],axis='columns')
df_train_merged_final.head(2)
# df_train_merged_final.columns.tolist()

,keyword,location,text,target,caracteres_tweet,text_vector,text_vector_filtrado,elem_vector,elem_vector_filtrado,ablaze,accident,aftershock,airplane_accident,ambulance,annihilated,annihilation,apocalypse,armageddon,army,arson,arsonist,attack,attacked,avalanche,battle,bioterror,bioterrorism,blaze,blazing,bleeding,blew_up,blight,blizzard,blood,bloody,blown_up,body_bag,body_bagging,body_bags,bomb,...,snowstorm,storm,stretcher,structural_failure,suicide_bomb,suicide_bomber,suicide_bombing,sunk,survive,survived,survivors,terrorism,terrorist,threat,thunder,thunderstorm,tornado,tragedy,trapped,trauma,traumatised,trouble,tsunami,twister,typhoon,upheaval,violent_storm,volcano,war_zone,weapon,weapons,whirlwind,wild_fires,wildfire,windstorm,wounded,wounds,wreck,wreckage,wrecked
0,sin keyword,sin location,Our Deeds are the Reason of this #earthquake M...,1,69,"[our, deeds, are, the, reason, of, this, earth...","[deeds, reason, earthquake, may, allah, forgiv...",13,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,sin keyword,sin location,Forest fire near La Ronge Sask. Canada,1,38,"[forest, fire, near, la, ronge, sask, canada]","[forest, fire, near, la, ronge, sask, canada]",7,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

In [338]:
X=df_train_merged_final.drop(df_train_merged_final.columns[[0,1,2,4,5,6,7,8]], axis='columns')
X.head(2)

,target,ablaze,accident,aftershock,airplane_accident,ambulance,annihilated,annihilation,apocalypse,armageddon,army,arson,arsonist,attack,attacked,avalanche,battle,bioterror,bioterrorism,blaze,blazing,bleeding,blew_up,blight,blizzard,blood,bloody,blown_up,body_bag,body_bagging,body_bags,bomb,bombed,bombing,bridge_collapse,buildings_burning,buildings_on_fire,burned,burning,burning_buildings,...,snowstorm,storm,stretcher,structural_failure,suicide_bomb,suicide_bomber,suicide_bombing,sunk,survive,survived,survivors,terrorism,terrorist,threat,thunder,thunderstorm,tornado,tragedy,trapped,trauma,traumatised,trouble,tsunami,twister,typhoon,upheaval,violent_storm,volcano,war_zone,weapon,weapons,whirlwind,wild_fires,wildfire,windstorm,wounded,wounds,wreck,wreckage,wrecked
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [340]:
y=df_train_merged_final.target
y

0       1
1       1
2       1
3       1
4       1
       ..
7604    1
7605    1
7606    1
7608    1
7612    1
Name: target, Length: 7503, dtype: int64

##### Exporto 'X' con las variables dummies a .xlsx para verificar ejecucin ####

In [0]:
X.to_excel('X_dummy.xlsx')

#### Verifico el modelo de regresión lineal con las variables X,y

In [341]:
model.fit(X,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [0]:
# model.predict([[1,0,0]]) --> ver error en los parametros

In [344]:
# ver modelo de scoring
model.score(X,y)

1.0

In [345]:
df_train_merged_final.head(2)

,keyword,location,text,target,caracteres_tweet,text_vector,text_vector_filtrado,elem_vector,elem_vector_filtrado,ablaze,accident,aftershock,airplane_accident,ambulance,annihilated,annihilation,apocalypse,armageddon,army,arson,arsonist,attack,attacked,avalanche,battle,bioterror,bioterrorism,blaze,blazing,bleeding,blew_up,blight,blizzard,blood,bloody,blown_up,body_bag,body_bagging,body_bags,bomb,...,snowstorm,storm,stretcher,structural_failure,suicide_bomb,suicide_bomber,suicide_bombing,sunk,survive,survived,survivors,terrorism,terrorist,threat,thunder,thunderstorm,tornado,tragedy,trapped,trauma,traumatised,trouble,tsunami,twister,typhoon,upheaval,violent_storm,volcano,war_zone,weapon,weapons,whirlwind,wild_fires,wildfire,windstorm,wounded,wounds,wreck,wreckage,wrecked
0,sin keyword,sin location,Our Deeds are the Reason of this #earthquake M...,1,69,"[our, deeds, are, the, reason, of, this, earth...","[deeds, reason, earthquake, may, allah, forgiv...",13,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,sin keyword,sin location,Forest fire near La Ronge Sask. Canada,1,38,"[forest, fire, near, la, ronge, sask, canada]","[forest, fire, near, la, ronge, sask, canada]",7,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
x_train, x_test, y_train, y_test = train_test_split(df_train_merged_final['text_vector_filtrado'], df_train_merged_final['target'], test_size=0.2, random_state=4)

In [0]:
count_vector=CountVectorizer()

In [357]:
#documents=df_train_merged_final['text_vector_filtrado']
#documents

0       [deeds, reason, earthquake, may, allah, forgiv...
1           [forest, fire, near, la, ronge, sask, canada]
2       [residents, asked, shelter, place, notified, o...
3       [13000, people, receive, wildfires, evacuation...
4       [got, sent, photo, ruby, alaska, smoke, wildfi...
                              ...                        
7604    [worldnews, fallen, powerlines, glink, tram, u...
7605    [flip, side, im, walmart, bomb, everyone, evac...
7606    [suicide, bomber, kills, 15, saudi, security, ...
7608    [two, giant, cranes, holding, bridge, collapse...
7612    [latest, homes, razed, northern, california, w...
Name: text_vector_filtrado, Length: 7503, dtype: object

In [359]:
#x_traincv=cv.fit_transform([df_train_merged_final['text']])
#count_vector.fit(documents)

TypeError: ignored

In [363]:
d=df_train_merged_final['text_vector_filtrado'].array
d

<PandasArray>
[                                                                                   ['deeds', 'reason', 'earthquake', 'may', 'allah', 'forgive', 'us'],
                                                                                           ['forest', 'fire', 'near', 'la', 'ronge', 'sask', 'canada'],
                            ['residents', 'asked', 'shelter', 'place', 'notified', 'officers', 'evacuation', 'shelter', 'place', 'orders', 'expected'],
                                                                     ['13000', 'people', 'receive', 'wildfires', 'evacuation', 'orders', 'california'],
                                                                   ['got', 'sent', 'photo', 'ruby', 'alaska', 'smoke', 'wildfires', 'pours', 'school'],
                    ['rockyfire', 'update', 'california', 'hwy', '20', 'closed', 'directions', 'due', 'lake', 'county', 'fire', 'cafire', 'wildfires'],
                           ['flood', 'disaster', 'heavy', 'rain', 'causes'

In [0]:
#documentos=d

In [0]:
#x_traincv=cv.fit_transform([d])
#count_vector.fit(d)

In [0]:
# 2.Categorizo 'keyword'

In [334]:
pd.get_dummies(df_train['keyword'], drop_first=True).shape

(7503, 221)

In [335]:
# Vemos el top 20 de las categorias mas frecuentes para la serie keyword 
df_train['keyword'].value_counts().sort_values(ascending=False).head(20)

sin keyword    56
fatalities     45
deluge         42
armageddon     42
harm           41
damage         41
body_bags      41
siren          40
fear           40
twister        40
evacuate       40
windstorm      40
weapons        39
sinkhole       39
explosion      39
sunk           39
wrecked        39
outbreak       39
famine         39
collision      39
Name: keyword, dtype: int64

In [336]:
df_train.head()

,keyword,location,text,target,caracteres_tweet,text_vector,text_vector_filtrado,elem_vector,elem_vector_filtrado,keyword_esp
0,sin keyword,sin location,Our Deeds are the Reason of this #earthquake M...,1,69,"[our, deeds, are, the, reason, of, this, earth...","[deeds, reason, earthquake, may, allah, forgiv...",13,7,sin_keyword
1,sin keyword,sin location,Forest fire near La Ronge Sask. Canada,1,38,"[forest, fire, near, la, ronge, sask, canada]","[forest, fire, near, la, ronge, sask, canada]",7,7,sin_keyword
2,sin keyword,sin location,All residents asked to 'shelter in place' are ...,1,133,"[all, residents, asked, to, shelter, in, place...","[residents, asked, shelter, place, notified, o...",22,11,sin_keyword
3,sin keyword,sin location,"13,000 people receive #wildfires evacuation or...",1,65,"[13000, people, receive, wildfires, evacuation...","[13000, people, receive, wildfires, evacuation...",8,7,sin_keyword
4,sin keyword,sin location,Just got sent this photo from Ruby #Alaska as ...,1,88,"[just, got, sent, this, photo, from, ruby, ala...","[got, sent, photo, ruby, alaska, smoke, wildfi...",16,9,sin_keyword


In [337]:
#pd.Series(df_train['keyword'].unique).to_dict()
df_train['keyword'].unique().tolist()

['sin keyword',
 'ablaze',
 'accident',
 'aftershock',
 'airplane_accident',
 'ambulance',
 'annihilated',
 'annihilation',
 'apocalypse',
 'armageddon',
 'army',
 'arson',
 'arsonist',
 'attack',
 'attacked',
 'avalanche',
 'battle',
 'bioterror',
 'bioterrorism',
 'blaze',
 'blazing',
 'bleeding',
 'blew_up',
 'blight',
 'blizzard',
 'blood',
 'bloody',
 'blown_up',
 'body_bag',
 'body_bagging',
 'body_bags',
 'bomb',
 'bombed',
 'bombing',
 'bridge_collapse',
 'buildings_burning',
 'buildings_on_fire',
 'burned',
 'burning',
 'burning_buildings',
 'bush_fires',
 'casualties',
 'casualty',
 'catastrophe',
 'catastrophic',
 'chemical_emergency',
 'cliff_fall',
 'collapse',
 'collapsed',
 'collide',
 'collided',
 'collision',
 'crash',
 'crashed',
 'crush',
 'crushed',
 'curfew',
 'cyclone',
 'damage',
 'danger',
 'dead',
 'death',
 'deaths',
 'debris',
 'deluge',
 'deluged',
 'demolish',
 'demolished',
 'demolition',
 'derail',
 'derailed',
 'derailment',
 'desolate',
 'desolation',
 